In [1]:
import scanpy as sc

In [2]:
ad_mana = sc.read_h5ad('../data/mana.h5ad')

In [3]:
ad_cc = sc.read_h5ad('../data/cellcharter.h5ad')

In [9]:
ad_mana

AnnData object with n_obs × n_vars = 107228 × 5101
    obs: 'cell', 'centroid_x', 'centroid_y', 'centroid_z', 'component', 'volume', 'surface_area', 'scale', 'region', 'sample_id', 'proseg_cluster', 'output_folder', 'Num', 'n_genes', 'n_counts', 'louvain_0.5', 'louvain_1', 'louvain_1.5', 'louvain_2', 'louvain_2.5', 'louvain_3', 'louvain_3.5', 'Cluster', 'Level1', 'Level2', 'Level3', 'Level3.1', 'grid_label', 'rbd_domain', 'rbd_domain_0.1', 'rbd_domain_0.2', 'rbd_domain_0.3', 'rbd_domain_0.5', 'rbd_domain_0.6', 'rbd_domain_0.7', 'rbd_domain_0.8', 'rbd_domain_0.9', 'rbd_domain_1.1', 'rbd_domain_1', 'rbd_domain_1.25', 'rbd_domain_1.4', 'rbd_domain_1.5', 'leiden_0.5', 'leiden_1', 'leiden_1.5', 'leiden_2', 'leiden_2.5', 'leiden_3', 'leiden_3.5', 'sample_name', 'course', 'condition', 'model', 'cytetype_annotation_louvain_3.5', 'cytetype_cellOntologyTerm_louvain_3.5', 'cluster_id', 'author_label', 'annotation', 'Class', 'state', 'CL_term', 'CL_term_id', 'confidence', 'author_label_similarity_

In [11]:
import numpy as np

def label_coherence(adata, labels_key, conn_key="spatial_connectivities"):
    A = adata.obsp[conn_key].tocsr()
    labels = adata.obs[labels_key].to_numpy()
    coh = np.zeros(adata.n_obs, dtype=float)

    for i in range(adata.n_obs):
        start, end = A.indptr[i], A.indptr[i+1]
        neigh = A.indices[start:end]
        if neigh.size == 0:
            coh[i] = np.nan
            continue
        coh[i] = np.mean(labels[neigh] == labels[i])
    return coh

coh_cc   = label_coherence(ad_mana, "CC_cluster")
coh_mana = label_coherence(ad_mana, "MANA_cluster")

np.nanmedian(coh_cc), np.nanmedian(coh_mana)

(np.float64(0.875), np.float64(0.7884615384615384))

In [13]:
import scipy.sparse as sp

In [14]:
def graph_cut_fraction(adata, labels_key, conn_key="spatial_connectivities"):
    A = adata.obsp[conn_key].tocsr()
    labels = adata.obs[labels_key].to_numpy()
    # Use upper triangle to avoid double counting
    A = sp.triu(A, k=1).tocsr()
    i, j = A.nonzero()
    return np.mean(labels[i] != labels[j])

cut_cc   = graph_cut_fraction(ad_mana, "CC_cluster")
cut_mana = graph_cut_fraction(ad_mana, "MANA_cluster")
cut_cc, cut_mana

(np.float64(0.2711521704050347), np.float64(0.2908141035985154))

In [ ]:
def graph_cut_fraction(adata, labels_key, conn_key="spatial_connectivities"):
    A = adata.obsp[conn_key].tocsr()
    labels = adata.obs[labels_key].to_numpy()
    # Use upper triangle to avoid double counting
    A = sp.triu(A, k=1).tocsr()
    i, j = A.nonzero()
    return np.mean(labels[i] != labels[j])

cut_cc   = graph_cut_fraction(ad, "cellcharter_cluster")
cut_mana = graph_cut_fraction(ad, "mana_cluster")
cut_cc, cut_mana